In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
import os

# Data loading

In [2]:
batch_size = 16
img_size = (int(1857), int(1317))

In [3]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory('chest_xray/train/', 
                                                               image_size=img_size, batch_size=batch_size, 
                                                               color_mode='rgb',)

test_ds = tf.keras.preprocessing.image_dataset_from_directory('chest_xray/test/', 
                                                               image_size=img_size, batch_size=batch_size, 
                                                              color_mode='rgb')
val_ds = tf.keras.preprocessing.image_dataset_from_directory('chest_xray/val/',
                                                             image_size=img_size, batch_size=1, 
                                                             color_mode='rgb')

Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


# Preprocessing

In [4]:
resize_factor = 5
resized_size = (int(img_size[0]/resize_factor), int(img_size[1]/resize_factor))

resize_and_rescale = Sequential([
  experimental.preprocessing.Resizing(resized_size[0], resized_size[1]),
  #experimental.preprocessing.Rescaling(1./255)
])

data_augmentation = Sequential(
    [
        experimental.preprocessing.RandomFlip("horizontal"),
        experimental.preprocessing.RandomRotation(factor=0.05),
        experimental.preprocessing.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)

In [5]:
pneuomnia_size = len(os.listdir('chest_xray/train/PNEUMONIA'))
normal_size = len(os.listdir('chest_xray/train/NORMAL'))

weight_0 = (1 / normal_size)*(pneuomnia_size+normal_size)/2.0 
weight_1 = (1 / pneuomnia_size)*(pneuomnia_size+normal_size)/2.0

class_weights = {0: np.round(weight_0, 2), 1: np.round(weight_1, 2)}
class_weights

{0: 1.94, 1: 0.67}

# Model creation

In [6]:
pretrained = tf.keras.applications.InceptionV3(
    include_top=False, 
    weights='imagenet',
    input_shape = resized_size + (3, )
)
pretrained.trainable = False

base_model = tf.keras.Model(pretrained.input, GlobalAveragePooling2D()(pretrained.output))

base_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 371, 263, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 185, 131, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 185, 131, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 185, 131, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [8]:
inputs = Input(shape=img_size + (3, ))
x = resize_and_rescale(inputs)
x = data_augmentation(x)
x = tf.keras.applications.inception_v3.preprocess_input(x)
x = base_model(x, training=False)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)
x = Dense(256)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1857, 1317, 3)]   0         
_________________________________________________________________
sequential (Sequential)      (None, 371, 263, 3)       0         
_________________________________________________________________
data_augmentation (Sequentia (None, 371, 263, 3)       0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 371, 263, 3)       0         
_________________________________________________________________
tf.math.subtract_1 (TFOpLamb (None, 371, 263, 3)       0         
_________________________________________________________________
model (Functional)           (None, 2048)              21802784  
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1049

In [9]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', 
    patience = 3, 
    verbose=1,
    factor=0.5, 
    min_lr=0.00001)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    'inception_model_checkpoints',
    monitor="val_accuracy",
    save_best_only=True,
)

In [10]:
model.compile(
    loss='binary_crossentropy', 
    optimizer=tf.keras.optimizers.Adam(), 
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

In [11]:
h = model.fit(
    train_ds, 
    epochs=10, 
    validation_data=test_ds, 
    callbacks=[early_stopping, learning_rate_reduction, checkpoint_callback],
    class_weight=class_weights
)

Epoch 1/10
326/326 [==============================] - 390s 1s/step - loss: 0.4507 - accuracy: 0.7823 - precision: 0.9504 - recall: 0.7450 - val_loss: 0.6374 - val_accuracy: 0.7436 - val_precision: 0.8966 - val_recall: 0.6667
INFO:tensorflow:Assets written to: inception_model_checkpoints/assets
Epoch 2/10
326/326 [==============================] - 339s 1s/step - loss: 0.2457 - accuracy: 0.8861 - precision: 0.9686 - recall: 0.8757 - val_loss: 0.4179 - val_accuracy: 0.8349 - val_precision: 0.8021 - val_recall: 0.9769
INFO:tensorflow:Assets written to: inception_model_checkpoints/assets
Epoch 3/10
326/326 [==============================] - 341s 1s/step - loss: 0.2212 - accuracy: 0.9045 - precision: 0.9723 - recall: 0.8970 - val_loss: 0.3398 - val_accuracy: 0.8782 - val_precision: 0.8720 - val_recall: 0.9436
INFO:tensorflow:Assets written to: inception_model_checkpoints/assets
Epoch 4/10
326/326 [==============================] - 336s 1s/step - loss: 0.2060 - accuracy: 0.9185 - precision: 0

# Fine tuning the Inception base model by unfreezing the last convolutional block

In [19]:
#model.load_weights('inception_model_checkpoints/variables/variables')

In [13]:
start_finetuning_from = None

for i in range(len(pretrained.layers)):
    if pretrained.layers[i].name == 'conv2d_89':
        start_finetuning_from = i
        break

print('Start finetuning from layer ' + str(start_finetuning_from))

for layer in pretrained.layers[start_finetuning_from:]:
    layer.trainable = True
for layer in pretrained.layers[:start_finetuning_from]:
    layer.trainable = False

model.summary()

Start finetuning from layer 280
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1857, 1317, 3)]   0         
_________________________________________________________________
sequential (Sequential)      (None, 371, 263, 3)       0         
_________________________________________________________________
data_augmentation (Sequentia (None, 371, 263, 3)       0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 371, 263, 3)       0         
_________________________________________________________________
tf.math.subtract_1 (TFOpLamb (None, 371, 263, 3)       0         
_________________________________________________________________
model (Functional)           (None, 2048)              21802784  
_________________________________________________________________
dense_1 (Dense)            

In [14]:
model.compile(
    loss='binary_crossentropy', 
    optimizer=tf.keras.optimizers.Adam(1e-6), 
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

finetune_h = model.fit(
    train_ds, 
    epochs=5, 
    validation_data=test_ds, 
    callbacks=[early_stopping, checkpoint_callback],
    class_weight=class_weights
)

Epoch 1/5
326/326 [==============================] - 389s 1s/step - loss: 0.1816 - accuracy: 0.9278 - precision_1: 0.9803 - recall_1: 0.9215 - val_loss: 0.2685 - val_accuracy: 0.8974 - val_precision_1: 0.9158 - val_recall_1: 0.9205
Epoch 2/5
326/326 [==============================] - 339s 1s/step - loss: 0.1718 - accuracy: 0.9245 - precision_1: 0.9803 - recall_1: 0.9168 - val_loss: 0.2600 - val_accuracy: 0.8942 - val_precision_1: 0.9091 - val_recall_1: 0.9231
Epoch 3/5
326/326 [==============================] - 337s 1s/step - loss: 0.1641 - accuracy: 0.9358 - precision_1: 0.9800 - recall_1: 0.9329 - val_loss: 0.2518 - val_accuracy: 0.8990 - val_precision_1: 0.9247 - val_recall_1: 0.9128
Epoch 4/5
326/326 [==============================] - 339s 1s/step - loss: 0.1621 - accuracy: 0.9350 - precision_1: 0.9826 - recall_1: 0.9291 - val_loss: 0.2467 - val_accuracy: 0.8990 - val_precision_1: 0.9098 - val_recall_1: 0.9308
Epoch 5/5
326/326 [==============================] - 337s 1s/step - loss

In [15]:
model.fit(
    train_ds, 
    epochs=15, 
    validation_data=test_ds, 
    callbacks=[early_stopping, checkpoint_callback],
    class_weight=class_weights,
    initial_epoch=5
)

Epoch 6/15
326/326 [==============================] - 336s 1s/step - loss: 0.1651 - accuracy: 0.9363 - precision_1: 0.9802 - recall_1: 0.9332 - val_loss: 0.2492 - val_accuracy: 0.9006 - val_precision_1: 0.9249 - val_recall_1: 0.9154
Epoch 7/15
326/326 [==============================] - 337s 1s/step - loss: 0.1490 - accuracy: 0.9387 - precision_1: 0.9808 - recall_1: 0.9357 - val_loss: 0.2446 - val_accuracy: 0.9087 - val_precision_1: 0.9280 - val_recall_1: 0.9256
INFO:tensorflow:Assets written to: inception_model_checkpoints/assets
Epoch 8/15
326/326 [==============================] - 337s 1s/step - loss: 0.1644 - accuracy: 0.9337 - precision_1: 0.9807 - recall_1: 0.9290 - val_loss: 0.2373 - val_accuracy: 0.9103 - val_precision_1: 0.9217 - val_recall_1: 0.9359
INFO:tensorflow:Assets written to: inception_model_checkpoints/assets
Epoch 9/15
326/326 [==============================] - 338s 1s/step - loss: 0.1562 - accuracy: 0.9402 - precision_1: 0.9824 - recall_1: 0.9363 - val_loss: 0.2396 

Unfortunately, finetuning gaves only 1% of improvement on test set

# Further fine tuning by unfreezing one more conv. block of Inception

In [18]:
start_finetuning_from = None

for i in range(len(pretrained.layers)):
    if pretrained.layers[i].name == 'conv2d_80':
        start_finetuning_from = i
        break

print('Start finetuning from layer ' + str(start_finetuning_from))

for layer in pretrained.layers[start_finetuning_from:]:
    layer.trainable = True
for layer in pretrained.layers[:start_finetuning_from]:
    layer.trainable = False

model.summary()

Start finetuning from layer 249
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1857, 1317, 3)]   0         
_________________________________________________________________
sequential (Sequential)      (None, 371, 263, 3)       0         
_________________________________________________________________
data_augmentation (Sequentia (None, 371, 263, 3)       0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 371, 263, 3)       0         
_________________________________________________________________
tf.math.subtract_1 (TFOpLamb (None, 371, 263, 3)       0         
_________________________________________________________________
model (Functional)           (None, 2048)              21802784  
_________________________________________________________________
dense_1 (Dense)            

In [19]:
model.compile(
    loss='binary_crossentropy', 
    optimizer=tf.keras.optimizers.Adam(1e-8), 
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

finetune_h = model.fit(
    train_ds, 
    epochs=5, 
    validation_data=test_ds, 
    callbacks=[early_stopping, checkpoint_callback],
    class_weight=class_weights
)

Epoch 1/5
326/326 [==============================] - 390s 1s/step - loss: 0.1530 - accuracy: 0.9398 - precision_2: 0.9873 - recall_2: 0.9311 - val_loss: 0.2390 - val_accuracy: 0.9071 - val_precision_2: 0.9301 - val_recall_2: 0.9205
Epoch 2/5
326/326 [==============================] - 342s 1s/step - loss: 0.1715 - accuracy: 0.9302 - precision_2: 0.9794 - recall_2: 0.9256 - val_loss: 0.2379 - val_accuracy: 0.9087 - val_precision_2: 0.9258 - val_recall_2: 0.9282
Epoch 3/5
326/326 [==============================] - 341s 1s/step - loss: 0.1629 - accuracy: 0.9351 - precision_2: 0.9794 - recall_2: 0.9325 - val_loss: 0.2381 - val_accuracy: 0.9087 - val_precision_2: 0.9302 - val_recall_2: 0.9231
Epoch 4/5
326/326 [==============================] - 341s 1s/step - loss: 0.1538 - accuracy: 0.9398 - precision_2: 0.9839 - recall_2: 0.9342 - val_loss: 0.2381 - val_accuracy: 0.9087 - val_precision_2: 0.9302 - val_recall_2: 0.9231
Epoch 5/5
326/326 [==============================] - 341s 1s/step - loss

:( no improvement

In [22]:
_, test_acc, test_prec, test_rec = model.evaluate(test_ds, verbose=False)
_, val_acc, val_prec, val_rec = model.evaluate(val_ds, verbose=False)

print('Final metrics')

print('Test set:')
print(f'Accuracy: {np.round(test_acc, 2)}; precision: {np.round(test_prec, 2)}; recall: {np.round(test_rec, 2)}')

print('\nValidation set:')
print(f'Accuracy: {np.round(val_acc, 2)}; precision: {np.round(val_prec, 2)}; recall: {np.round(val_rec, 2)}')

Final metrics
Test set:
Accuracy: 0.91; precision: 0.93; recall: 0.93

Validation set:
Accuracy: 0.62; precision: 0.6; recall: 0.75
